In [1]:
#imports
import sys
import pandas as pd
import pathlib
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
import pprint
import json
import api_setup
from sklearn.preprocessing import OneHotEncoder

In [5]:
CWD = pathlib.Path.cwd()
REPO_ROOT = CWD.parent.parent.parent
DATA_DIR = REPO_ROOT / "data"
APP = 'spotify'
NAME = 'nick'
SUBDIRS = 'my_spotify_data/MyData/'
USER_DATA_DIR = DATA_DIR / APP / NAME / SUBDIRS

# API Keys
sys.path.append(REPO_ROOT)
from api_setup import parse_api_kvs
# Understanding the spotipy API and making sure it's installed correctly
env_vars = api_setup.parse_api_kvs(REPO_ROOT / "api-keys")
spotify = spotipy.Spotify(client_credentials_manager=SpotifyOAuth(**env_vars))

In [13]:
def addFeatures(uri):
    audioFeatures = spotify.audio_features(uri)
    audioFeatures = pd.DataFrame(audioFeatures)
    audioFeatures = audioFeatures.drop(columns = ['type','id','uri','track_href','analysis_url','mode',
                                                  'duration_ms','time_signature'])
    return audioFeatures.iloc[0]

In [16]:
addFeatures('4rKEmhNA19JezqVsSQS4yo')

danceability          0.7380
energy                0.9540
key                   1.0000
loudness             -2.1460
speechiness           0.0356
acousticness          0.0692
instrumentalness      0.0000
liveness              0.2220
valence               0.7420
tempo               102.0150
Name: 0, dtype: float64

In [36]:
def combineMasterFeatures(masterTable):
    master = pd.read_csv(masterTable)
    #master = master.head(10) #remove this line later
    master = master.drop(['Unnamed: 0'], axis = 1) #drop extra index
    master['uri'] = master['uri'].str.replace('spotify:track:','')
    audioFeaturesDF = pd.DataFrame()
    masterFeatures = pd.concat([master, master.apply(lambda x: addFeatures(x['uri']), axis = 1)], axis = 1)
    masterFeatures = masterFeatures.drop(columns=['track','uri'])
    return masterFeatures

In [44]:
masterFeatures = combineMasterFeatures('masterTable.csv')
masterFeatures.head(10)

,artist,nick_history,jojo_history,rich_history,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,LOONA/yyxy,NaN,True,NaN,0.738,0.954,1.0,-2.146,0.0356,0.06920,0.000000,0.222,0.742,102.015
1,Danny Brown,NaN,True,NaN,0.690,0.600,8.0,-7.331,0.2490,0.18700,0.000000,0.694,0.761,160.080
2,Latto,NaN,True,NaN,0.790,0.614,9.0,-6.958,0.0991,0.06060,0.000000,0.239,0.630,145.907
3,MIMIDEATH,NaN,True,NaN,0.828,0.764,2.0,-7.361,0.1040,0.17000,0.309000,0.237,0.289,110.987
4,Rico Nasty,NaN,True,NaN,0.936,0.523,1.0,-6.390,0.2690,0.00365,0.000000,0.105,0.278,130.014
5,DaBaby,True,True,NaN,0.768,0.652,1.0,-2.708,0.3070,0.11300,0.000000,0.107,0.777,154.187
6,Otoboke Beaver,NaN,True,NaN,0.321,0.973,9.0,-1.191,0.1540,0.00751,0.684000,0.086,0.574,132.201
7,Flo Milli,NaN,True,NaN,0.782,0.844,10.0,-4.142,0.2300,0.10700,0.000001,0.295,0.760,77.435
8,"Tyler, The Creator",NaN,True,NaN,0.443,0.576,1.0,-8.695,0.2010,0.07090,0.019700,0.410,0.127,151.835
9,Lorde,NaN,True,NaN,0.733,0.620,1.0,-10.525,0.0750,0.60600,0.305000,0.117,0.247,111.039
